In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data_given/winequality.csv')
df

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
fixed_acidity,1599.0,8.319637,1.741096,4.60000,7.1000,7.90000,9.200000,15.90000
volatile_acidity,1599.0,0.527821,0.179060,0.12000,0.3900,0.52000,0.640000,1.58000
citric_acid,1599.0,0.270976,0.194801,0.00000,0.0900,0.26000,0.420000,1.00000
residual_sugar,1599.0,2.538806,1.409928,0.90000,1.9000,2.20000,2.600000,15.50000
chlorides,1599.0,0.087467,0.047065,0.01200,0.0700,0.07900,0.090000,0.61100
free_sulfur_dioxide,1599.0,15.874922,10.460157,1.00000,7.0000,14.00000,21.000000,72.00000
total_sulfur_dioxide,1599.0,46.467792,32.895324,6.00000,22.0000,38.00000,62.000000,289.00000
density,1599.0,0.996747,0.001887,0.99007,0.9956,0.99675,0.997835,1.00369
pH,1599.0,3.311113,0.154386,2.74000,3.2100,3.31000,3.400000,4.01000
sulphates,1599.0,0.658149,0.169507,0.33000,0.5500,0.62000,0.730000,2.00000


In [6]:
df.describe().loc[["min","max"]].T

,min,max
fixed_acidity,4.60000,15.90000
volatile_acidity,0.12000,1.58000
citric_acid,0.00000,1.00000
residual_sugar,0.90000,15.50000
chlorides,0.01200,0.61100
free_sulfur_dioxide,1.00000,72.00000
total_sulfur_dioxide,6.00000,289.00000
density,0.99007,1.00369
pH,2.74000,4.01000
sulphates,0.33000,2.00000


In [15]:
overview = df.describe()
overview.loc[["min","max"]].to_json("schema_in.json")


In [19]:
class NotInRange(Exception):
    def __init__(self,message = "value not in range"):
        self.message = message
        super().__init__(self.message)
    

In [20]:
raise NotInRange

NotInRange: value not in range

In [3]:
pwd

'C:\\Projets_Wistron\\VS_Projects\\wine_app\\notebooks'

In [7]:
from numpy import product
from src.get_data import read_params
import argparse
import mlflow
from mlflow.tracking import MlflowClient, client
from pprint import pprint
import joblib
import os

#os.chdir("C:\\Projets_Wistron\\VS_Projects\\wine_app\\notebook")

def log_production_model(config_path):
    config = read_params(config_path)
    mlflow_congig = config["mlflow_config"]
    model_name = mlflow_congig["registered_model_name"]
    
    remote_server_uri = mlflow_congig["remote_server_uri"]
    mlflow.set_registry_uri(remote_server_uri)

    runs = mlflow.search_runs(experiment_ids=1)
    lowest = runs["metrics.mae"].sort_values(ascending=True)[0]
    lowest_run_id = runs[runs["metrics.mae"] == lowest]["run_id"][0]

    client = MlflowClient()
    for mv in client.search_model_versions(f"name='{model_name}'"):
        mv = dict(mv)
        if mv["run_id"] == lowest_run_id:
            current_version = mv["version"]
            logged_model = mv["source"]
            pprint(mv,indent=4)
            client.transition_model_version_stage(
                name = model_name,
                version = current_version,
                stage = "Production"
            )
        else:
            current_version = mv["version"]
            client.transition_model_version_stage(
                name = model_name,
                version = current_version,
                stage = "Staging"
            )
            
log_production_model("../params.yaml")

TypeError: join() argument must be str or bytes, not 'int'

In [1]:
from src.get_data import read_params
import argparse
import mlflow
from mlflow.tracking import MlflowClient
from pprint import pprint
import joblib
import os



def log_production_model(config_path):
    config = read_params(config_path)
    
    
    mlflow_config = config["mlflow_config"] 
    

    model_name = mlflow_config["registered_model_name"]


    remote_server_uri = mlflow_config["remote_server_uri"]

    mlflow.set_tracking_uri(remote_server_uri)
    
    
    runs = mlflow.search_runs(experiment_ids=1)
    lowest = runs["metrics.mae"].sort_values(ascending=True)[0]
    lowest_run_id = runs[runs["metrics.mae"] == lowest]["run_id"][0]
    

    client = MlflowClient()
    for mv in client.search_model_versions(f"name='{model_name}'"):
        mv = dict(mv)
        
        if mv["run_id"] == lowest_run_id:
            current_version = mv["version"]
            logged_model = mv["source"]
            pprint(mv, indent=4)
            client.transition_model_version_stage(
                name=model_name,
                version=current_version,
                stage="Production"
            )
        else:
            current_version = mv["version"]
            client.transition_model_version_stage(
                name=model_name,
                version=current_version,
                stage="Staging"
            )        

                
log_production_model("../params.yaml")

{   'creation_timestamp': 1625731392529,
    'current_stage': 'Production',
    'description': '',
    'last_updated_timestamp': 1625736263140,
    'name': 'ElasticNetWineModel',
    'run_id': '3023745efe77443899bf0e5fcdef59c4',
    'run_link': '',
    'source': './artifacts/1/3023745efe77443899bf0e5fcdef59c4/artifacts/model',
    'status': 'READY',
    'status_message': '',
    'tags': {},
    'user_id': '',
    'version': '3'}
